<a href="https://colab.research.google.com/github/anelglvz/Working-Analyst/blob/main/ML-AI-for-the-Working-Analyst/Semana%207/Semana7_2_Working_Analyst_Continuacion_SpectralClustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clusterización de trayectorias usando Clustering Espectral y KMeans

## Dependencias

In [ ]:
import urllib
import zipfile
import os
import scipy.io
import math

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.spatial.distance import directed_hausdorff
from scipy import linalg

from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

# Some visualization stuff, not so important
sns.set()
plt.rcParams['figure.figsize'] = (10, 10)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Funciones

In [ ]:
def plot_cluster(traj_lst, cluster_lst):
    cluster_count = np.max(cluster_lst) + 1
    
    for traj, cluster in zip(traj_lst, cluster_lst):
        plt.plot(traj[:, 0], traj[:, 1], c=plt.cm.tab20(cluster))
    plt.show()

## Obtención de los datos

Trabajaremos con el dataset LABOMNI que contiene trayectorias de humanos caminando por un laboratorio capturadas usando una cámara omni-direccional.

In [ ]:
# link descontinuado 'http://cvrr.ucsd.edu/LISA/Datasets/TrajectoryClustering/CVRR_dataset_trajectory_clustering.zip', era el oficial

filename = '/content/drive/MyDrive/Curso-WorkingAnalyst/semana7/labomni.mat'

###### Función util si aún hay link, descarga un .zip
# is_download_required = not os.path.exists(data_folder)

#if is_download_required:
#    zip_filename = 'data.zip'
#    urllib.request.urlretrieve(dataset_link, zip_filename)
#    zip_ref = zipfile.ZipFile(zip_filename, 'r')
#    zip_ref.extractall(data_folder)
#    zip_ref.close()

# Import dataset
traj_data = scipy.io.loadmat(filename)['tracks']

traj_lst = []
for data_instance in traj_data:
    traj_lst.append(np.vstack(data_instance[0]).T)


In [ ]:
raw_data = scipy.io.loadmat(filename)

In [ ]:
raw_data['tracks'][0][0].T

In [ ]:
# La longitud de la lista es la cantidad de caminos
traj_lst

In [ ]:
traj_lst[0].shape

## Visualización de los datos

In [ ]:
# Plotting

for traj in traj_lst:
    plt.plot(traj[:, 0], traj[:, 1])

In [ ]:
plt.plot(traj_lst[157][:,0],traj_lst[157][:,1])
plt.plot(traj_lst[193][:,0],traj_lst[193][:,1])

# Exploración de los datos



In [ ]:
# auxiliar
lenghts = [len(i) for i in traj_lst]

array = np.array(lenghts)

In [ ]:
# Pequeño ejercicio: 
# Encontrar los conjuntos de puntos mas grande y el mas pequeño 
#                        Pista: np.where()
index = np.where(array == array.max())[0][0]   
print(index)
traj_lst[index]

In [ ]:
index = np.argmax(array)
index

In [ ]:
len(traj_lst[index])

In [ ]:
index2 = np.where(array == array.min())[0][0]
print(index2)
len(traj_lst[index2])

In [ ]:
index2 = np.argmin(array)
index2

In [ ]:
# En realidad, ¿que son nuestros datos?
plt.scatter(traj_lst[193][:,0], traj_lst[193][:,1])

# Distancia de Hausdorff

Para calcular la similitud entre dos trayectorias utilizaremos la distancia de Hausdorff programada en [```scipy.spatial.distance.directed_hausdorff```](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.directed_hausdorff.html). .

In [ ]:
# ¿Resulta familiar?
punto_a = np.array([[0,0]])
punto_b = np.array([[3,4]])
directed_hausdorff(punto_b, punto_a)

In [ ]:
# ¿Y ahora?
conj_a = np.array([[1,1],[0,0]])
conj_b = np.array([[1,1]])
directed_hausdorff(conj_a, conj_b)

In [ ]:
# ¿Y ahora?
conj_a = np.array([[1,1],[0,0]])
conj_b = np.array([[1,1]])
directed_hausdorff(conj_b, conj_a)

In [ ]:
# Intente calcular la distancia de hausdorff entre 2 elementos y vea gráficamente que ocurre

## Matriz de similitud/distancia/adyacencia

In [ ]:
%%time
def hausdorff( u, v):
    d = max(directed_hausdorff(u, v)[0], directed_hausdorff(v, u)[0])
    return d

traj_count = len(traj_lst)
A = np.zeros((traj_count, traj_count))

# This may take a while
for i in range(traj_count):
    for j in range(traj_count):
        distance = hausdorff(traj_lst[i], traj_lst[j])
        A[i, j] = distance
        

In [ ]:
print(A.shape)
A[:10, :4]

In [ ]:
# PENDIENTE (minimo sin contar los ceros)
np.min(A[A > 0])

## Clustering Espectral

Obtenemos la $D$ que es una matriz diagonal en la que tendremos el ```grado``` de la trayectoría $i$ en la $i-ésima$ entrada.

$D_{i} = \sum_j A_{i, j}$

In [ ]:
D_vect = np.sum(A,axis = 1) # Suma por renglones
D_vect

In [ ]:
D = np.diag(D_vect) # matriz diagonal de un vector
D[:10, :4]

Calculamos la matriz $L$. No es la matriz Laplaciana de $A$, pero esta relacionada.

$L = D^{-1/2}~A~D^{-1/2}$

In [ ]:
D_sqrtinv = np.diag(np.sqrt(1/D_vect))
L_norm = D_sqrtinv @ A @ D_sqrtinv
L_norm[:10, :4]

In [ ]:
L_norm.shape

Hacemos la descomposición en valores singulares 

$L = U~\Sigma~V^T$.

Los eigenvectores van son las columnas de $U$.

In [ ]:
U, sigma, _ = linalg.svd(L_norm, full_matrices=False)
print(U.shape, sigma.shape)
sigma[:10]

In [ ]:
np.max(L_norm - U @ np.diag(sigma) @ U.T)

In [ ]:
suma = np.sum(sigma)
suma_parcial = np.sum(sigma[:15])
suma_parcial/suma

In [ ]:
sigma[200]

Definimos $k$ como el número de clusters que queremos crear y nos *quedamos* con los $k$ eigenvectores correspondientes a los $k$ eigenvalores más grandes.

In [ ]:
j = 15
Uj = U[:, :j]
Uj.shape

Finalmente, normalizamos cada fila de la matriz resultante, $U_k$, y ocupamos éstas como *feature vectors* para KMeans.

In [ ]:
k = 15
y_pred = KMeans(n_clusters = k).fit_predict(normalize(Uj))
y_pred

In [ ]:
y_pred = y_pred + 1

In [ ]:
y_pred.shape

In [ ]:
plot_cluster(traj_lst, y_pred)

In [ ]:
cluster_n = 4
plot_cluster([traj_lst[i] for i in np.where(y_pred == cluster_n)[0]], y_pred[y_pred == cluster_n])

In [ ]:
y_pred

In [ ]:
unique, counts = np.unique(y_pred, return_counts=True)
print(np.asarray((unique, counts)).T)

### Ejercicios

- En este notebook esta implementado el algoritmo de Ng, Jordan y Weiss (NJW). Implementar cualquier otro algoritmo ([referencia](https://sites.stat.washington.edu/spectral/papers/UW-CSE-03-05-01.pdf))
- Comparar los resultados obtenidos con los verdaderos clusters que se encuentran en ```scipy.io.loadmat(filename)['truth']``` (pistas abajo)


In [ ]:
y_true = scipy.io.loadmat(filename)['truth']

In [ ]:
y_true.reshape(1,-1)

In [ ]:
true_pd = pd.DataFrame([element[0] for element in y_true])

In [ ]:
true_pd.value_counts()

In [ ]:
y_pred

In [ ]:
comparative = pd.concat([true_pd, pd.DataFrame(y_pred, columns=[1])], axis=1)

In [ ]:
comparative.rename(columns = {0:'True', 1:'Predicted'}, inplace = True)

In [ ]:
comparative

In [ ]:
comparative[comparative['True'] == comparative['Predicted']]

Otro Ejercicio: ¿Porque creen que pasa lo anterior?

In [ ]:
for cluster_n in range(1,k+1):
  plot_cluster([traj_lst[i] for i in np.where(y_pred == cluster_n)[0]], y_pred[y_pred == cluster_n])

In [ ]:
y_aux = y_true.reshape(209,).copy()
y_aux

In [ ]:
for cluster_n in range(1,16):
  plot_cluster([traj_lst[i] for i in np.where(y_pred == cluster_n)[0]], y_pred[y_aux == cluster_n])

# Selección de número de clusters

Método "Elbow"

In [ ]:
!pip install yellowbrick

In [ ]:
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

La funciones "KelbowVisualizer" permite mas metricas aparte de la metrica "distortion" (distancias media de elementos a sus centroides). Permite la metrica "silhouette" que calcula la media del coeficiente Silhouette (medida de que tan bien está clasificado cada grupo), y "calinski_harabasz" que calcula el radio de dispersion entre clusters y dentro del cluster.


In [ ]:
codo = KElbowVisualizer(KMeans(random_state=3), k=(4,20))
codo.fit(normalize(Uj))
codo.poof()

Utilizando otra de las métricas

In [ ]:
codo = KElbowVisualizer(KMeans(random_state=3), k=(4,20), metric='silhouette')
codo.fit(normalize(Uj))
codo.poof()

El coeficiente de Silhouette evalúa la cercania entre puntos de un mismo cluster al mismo tiempo que la distancia a los clusters vecinos.

El coeficiente de Silhouette oscila entre -1 y 1 donde 1 indica un clúster de alta densidad y -1 que no existe densidad alguna.

In [ ]:
model = KMeans(12)
graf_sil = SilhouetteVisualizer(model)
graf_sil.fit(normalize(Uj))
graf_sil.poof()

# Que pasaría si nos quedamos con mas características representativas

In [ ]:
j = 13
Uj = U[:, :j]
Uj.shape

In [ ]:
codo = KElbowVisualizer(KMeans(random_state=3), k=(4,20))
codo.fit(normalize(Uj))
codo.poof()

In [ ]:
codo = KElbowVisualizer(KMeans(), k=(4,20), metric='silhouette')
codo.fit(normalize(Uj))
codo.poof()

In [ ]:
model = KMeans(7)
graf_sil = SilhouetteVisualizer(model)
graf_sil.fit(normalize(Uj))
graf_sil.poof()